In [1]:
from heeps.util.multiCPU import multiCPU
from heeps.util.freq_decomp import fit_zer, remove_zernike
from heeps.util.img_processing import resize_cube
import numpy as np
import os  
from astropy.io import fits
from copy import deepcopy
import proper

In [2]:
os.chdir(os.path.normpath(os.path.expandvars('$HOME/heeps_metis/input_files/wavefront')))
filename = 'cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_%s_ncpa_%s_%s.fits'
tag = {'L': 'LM', 'N2': 'N'}
rep = {'L': 'rep_6_-0h30', 'N2': 'rep_5_-0h30'}
npupils = {'L': 285, 'N2': 119}
sigLFs = {'L': 13e-9, 'N2': 13e-9*2}
sigHFs = {'L': 3e-9, 'N2': 3e-9*2}
nzer = 100
G = 0.4
freq = 3
lag = 3

In [ ]:
def load_ncpa(case, band, samp=300, rms=0):
    npupil = npupils[band]
    if samp == 300:
        scao = fits.getdata('cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_scao_only_%s_%s.fits'%(band, npupil))
        cbw = fits.getdata('cbw/20221006/ncpa/%s_%s.fits'%(tag[band], rep[band]))
    elif samp == 100:
        scao = fits.getdata('cfull/cube_Cfull_20220929_3600s_100ms_scao_only_%s_%s.fits'%(band, npupil))
        cbw = np.repeat(fits.getdata('cbw/20221006/ncpa/%s_%s.fits'%(tag[band], rep[band])), repeats=3, axis=0)
    pup = fits.getdata('cfull/mask_Cfull_20220512_%s_%s.fits'%(band, npupil))
    pup[pup < .5] = 0
    if 'cbw' in case:
        ncpa = scao + cbw
    else:
        if rms == 0:
            wv = fits.getdata('wv/cube_Cbasic_20210601_3600s_%sms_0piston_meters_scao_only_%s_%s_WVonly.fits'%(samp, band, npupil))
        else:
            wv = fits.getdata('wv/cube_Cbasic_20210601_3600s_%sms_0piston_meters_scao_only_%s_%s_WVonly_rms_%s.fits'%(samp, band, npupil, rms))
        if 'wv' in case:
            ncpa = scao + wv
        else: # ALL NCPA
            ncpa = scao + cbw + wv
    return pup, ncpa

def load_zpols(pup, ncpa, ncpa_name, npupil, nzer=nzer):
    zpols_name = ncpa_name[:-5] + '_zpols_%s.fits'%nzer
    try:
        zpols = fits.getdata(zpols_name)
        print('    getdata ' + zpols_name)
    except FileNotFoundError:
        print('    writeto ' + zpols_name)
        zpols = multiCPU(fit_zer, posargs=[pup, npupil/2, nzer], 
            posvars=[ncpa], case='get zpols')
        fits.writeto(zpols_name, np.float32(zpols))
    return zpols

def load_zpols_integ(pup, ncpa, ncpa_name, npupil, lag, freqLF, freqHF, sigLF, sigHF, nzer=nzer, G=0.4):
    if freqLF == freqHF:
        zpols_integ_name = ncpa_name[:-5] + '_zpols_%s_freq_%s_G_%s_lag_%s_sigLF_%s_sigHF_%s.fits'%(nzer, freqLF, G, lag, sigLF, sigHF)
    else:
        zpols_integ_name = ncpa_name[:-5] + '_zpols_%s_freqLF_%s_freqHF_%s_G_%s_lag_%s_sigLF_%s_sigHF_%s.fits'%(nzer, freqLF, freqHF, G, lag, sigLF, sigHF)
    if os.path.isfile(zpols_integ_name):
        zpols_integ = fits.getdata(zpols_integ_name)
        print('  getdata ' + zpols_integ_name)
    else:
        print('  write to ' + zpols_integ_name)
        zpols = load_zpols(pup, ncpa, ncpa_name, npupil, nzer=nzer)
        zpols_integ = np.zeros(zpols.shape)
        nframes = len(zpols)
        # piston
        zpols_integ[:,0] = zpols[:,0]
        # tip-tilt, and higher modes
        for m, freq, sig in zip([range(1,3), range(3,nzer)], [freqLF, freqHF], [sigLF, sigHF]):
            if m.start == 3 and m.stop > 3: # !!!!!! ---> True for modes above tip-tilt !!!!!! (comes from latest commit Jan 2025 CDe)
                for n in range(freq+lag, nframes, freq):
                    error = np.mean(zpols[n-freq-lag:n-lag,m] - zpols_integ[n-freq-lag:n-lag,m], 0) + np.random.normal(0, sig, (1, len(m))) 
                    zpols_integ[n:n+freq,m] = zpols_integ[n-1,m] + G*error
        fits.writeto(zpols_integ_name, np.float32(zpols_integ))
    return zpols_integ

def save_ncpa_cube(case, band, lag, freqLF, freqHF, nzer=nzer, filename=filename, samp=300, rms=1, sigLFs=sigLFs, sigHFs=sigHFs):
    npupil = npupils[band]
    ncpa_name = filename%(case, band, npupil)
    if os.path.isfile(ncpa_name):
        print('file already exists: ' + ncpa_name)
    else:
        print('write to ' + ncpa_name)
        pup, ncpa = load_ncpa(case, band, samp=samp, rms=rms)
        sigLF = sigLFs[band]
        sigHF = sigHFs[band]
        zpols_integ = load_zpols_integ(pup, ncpa, ncpa_name, npupil, lag, freqLF, freqHF, sigLF, sigHF, nzer=nzer)
        wf = proper.prop_begin(1, 1, npupil, 1) # initial wavefront
        _, HSF = multiCPU(remove_zernike, multi_out=True, verbose=True,
            posargs=[deepcopy(wf), pup],
            posvars=[ncpa, zpols_integ])
        if samp == 100 and '300ms' in ncpa_name:
            fits.writeto(ncpa_name, np.float32(HSF[::3]))
        else:
            fits.writeto(ncpa_name, np.float32(HSF))
    return ncpa_name

In [4]:
for case in ['wv', 'cbw', 'all']:
    for band in ['L', 'N2']:
        save_ncpa_cube(case, band, lag, freq, freq)

file already exists: cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_wv_ncpa_L_285.fits
file already exists: cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_wv_ncpa_N2_119.fits
file already exists: cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_cbw_ncpa_L_285.fits
file already exists: cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_cbw_ncpa_N2_119.fits
file already exists: cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_all_ncpa_L_285.fits
file already exists: cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_all_ncpa_N2_119.fits


# New WV simulations:
### - 100 modes 1HZ (+ QACITS 10HZ)
### - 20 modes 10HZ (+ QACITS 10HZ)

In [5]:
# 100ms sampling (SCAO and WV)
samp = 100

# scaling water vapor screens
band = 'N2'
npupil = npupils[band]
wv = fits.getdata('wv/cube_Cbasic_20210601_3600s_100ms_0piston_meters_scao_only_720_WV.fits')        
temporal_rms = 8814.1
for rms in [150, 600, 1200]:
    name_300 = 'cfull/cube_Cfull_20220929_3600s_%sms_%s_ncpa'%(300, 'wv') + '_rms_%s_%s_%s.fits'%(rms, band, npupil)
    wv_name = 'wv/cube_Cbasic_20210601_3600s_%sms_0piston_meters_scao_only_%s_%s_WVonly_rms_%s.fits'%(samp, band, npupil, rms)
    if os.path.isfile(name_300):
        print('file already exists: ' + name_300)
    else:
        print('write to ' + name_300)
        scaling = rms/temporal_rms
        wv_cube = resize_cube(wv, npupil)*scaling
        fits.writeto(wv_name, wv_cube)
        scao = fits.getdata('cfull/cube_Cfull_20220929_3600s_100ms_scao_only_%s_%s.fits'%(band, npupil))
        fits.writeto(name_300, (scao + wv_cube)[::3])

file already exists: cfull/cube_Cfull_20220929_3600s_300ms_wv_ncpa_rms_150_N2_119.fits
file already exists: cfull/cube_Cfull_20220929_3600s_300ms_wv_ncpa_rms_600_N2_119.fits
file already exists: cfull/cube_Cfull_20220929_3600s_300ms_wv_ncpa_rms_1200_N2_119.fits


### control lag, frequency

In [6]:
lag = 10 # 1s
freqLF = 1 # 10Hz
freqHF = {20: 1, # 10Hz
         100: 10} # 1Hz
filename = 'cfull/cube_Cfull_20220929_3600s_%sms'
        
for case in ['wv']:
    for lag in [10, 1]: 
        for nzer in [20, 100]:
            for rms in [150, 600, 1200]:
                name_100 = filename%(100) + '_%s_ncpa_rms_%s_lag_%s_nzer_%s_%s_%s.fits'%(case, rms, lag, nzer, band, npupil)
                name_300 = filename%(300) + '_%s_ncpa_rms_%s_lag_%s_nzer_%s_%s_%s.fits'%(case, rms, lag, nzer, band, npupil)
                if os.path.isfile(name_300):
                    print('file already exists: ' + name_300)
                else:
                    ncpa_name = filename%(100) + '_%s_ncpa' + '_rms_%s_lag_%s_nzer_%s'%(rms, lag, nzer) + '_%s_%s.fits'
                    save_ncpa_cube(case, band, lag, freqLF, freqHF[nzer], nzer=nzer, filename=ncpa_name, samp=100, rms=rms)
                    fits.writeto(name_300, fits.getdata(name_100)[::3],overwrite=True)
                    os.remove(name_100)

file already exists: cfull/cube_Cfull_20220929_3600s_300ms_wv_ncpa_rms_150_lag_10_nzer_20_N2_119.fits
file already exists: cfull/cube_Cfull_20220929_3600s_300ms_wv_ncpa_rms_600_lag_10_nzer_20_N2_119.fits
file already exists: cfull/cube_Cfull_20220929_3600s_300ms_wv_ncpa_rms_1200_lag_10_nzer_20_N2_119.fits
file already exists: cfull/cube_Cfull_20220929_3600s_300ms_wv_ncpa_rms_150_lag_10_nzer_100_N2_119.fits
file already exists: cfull/cube_Cfull_20220929_3600s_300ms_wv_ncpa_rms_600_lag_10_nzer_100_N2_119.fits
file already exists: cfull/cube_Cfull_20220929_3600s_300ms_wv_ncpa_rms_1200_lag_10_nzer_100_N2_119.fits
file already exists: cfull/cube_Cfull_20220929_3600s_300ms_wv_ncpa_rms_150_lag_1_nzer_20_N2_119.fits
file already exists: cfull/cube_Cfull_20220929_3600s_300ms_wv_ncpa_rms_600_lag_1_nzer_20_N2_119.fits
file already exists: cfull/cube_Cfull_20220929_3600s_300ms_wv_ncpa_rms_1200_lag_1_nzer_20_N2_119.fits
file already exists: cfull/cube_Cfull_20220929_3600s_300ms_wv_ncpa_rms_150_lag_

# 19/01/2025 CBW Simulations

In [7]:
filename = 'cfull/cube_Cfull_20250119_3600s_300ms_%s_ncpa_%s_%s.fits'
samp = 100 # ms
lag = 2
freq = 1 # 10Hz
freqLF = freq
freqHF = freq
sigLFs = {'L': 0, 'N2': 0}
sigHFs = {'L': 0, 'N2': 0}
for band in ['L', 'N2']:
    for nzer in [2, 20, 100]:
        case = 'nzer_%s_cbw'%nzer
        save_ncpa_cube(case, band, lag, freq, freq, nzer=nzer, samp=samp, filename=filename, sigLFs=sigLFs, sigHFs=sigHFs)

file already exists: cfull/cube_Cfull_20250119_3600s_300ms_nzer_2_cbw_ncpa_L_285.fits
write to cfull/cube_Cfull_20250119_3600s_300ms_nzer_20_cbw_ncpa_L_285.fits
  write to cfull/cube_Cfull_20250119_3600s_300ms_nzer_20_cbw_ncpa_L_285_zpols_20_freq_1_G_0.4_lag_2_sigLF_0_sigHF_0.fits
    getdata cfull/cube_Cfull_20250119_3600s_300ms_nzer_20_cbw_ncpa_L_285_zpols_20.fits
   2025-01-19 21:40:44, using 56 cores
   2025-01-19 21:49:27, completed in 523.02 seconds
write to cfull/cube_Cfull_20250119_3600s_300ms_nzer_100_cbw_ncpa_L_285.fits
  write to cfull/cube_Cfull_20250119_3600s_300ms_nzer_100_cbw_ncpa_L_285_zpols_100_freq_1_G_0.4_lag_2_sigLF_0_sigHF_0.fits
    writeto cfull/cube_Cfull_20250119_3600s_300ms_nzer_100_cbw_ncpa_L_285_zpols_100.fits
   2025-01-19 21:50:24, get zpols using 56 cores
   2025-01-19 23:55:19, completed in 7495.44 seconds
   2025-01-19 23:55:22, using 56 cores
   2025-01-20 00:30:46, completed in 2123.63 seconds
write to cfull/cube_Cfull_20250119_3600s_300ms_nzer_2_cbw_

# November 2025: New run
- reduced wv_rms: 310 baseline vs 600. Is it in unit of 10^-8 meters ??

In [ ]:
os.chdir(os.path.normpath(os.path.expandvars('$HOME/heeps_metis/input_files/wavefront')))
# filename = 'cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_%s_ncpa_%s_%s.fits'
tag = {'L': 'LM', 'N2': 'N'}
rep = {'L': 'rep_6_-0h30', 'N2': 'rep_5_-0h30'}
npupils = {'L': 285, 'N2': 119}
sigLFs = {'L': 13e-9, 'N2': 13e-9*2}
sigHFs = {'L': 3e-9, 'N2': 3e-9*2}

nzer = 20
lag = 2 # 2 frames at 100ms -> 200ms
G = 0.33



temporal_rms = 8814.1
wv_rms_N = 310
wv_rms_L = 310 / 12
samp = 100

band = 'N2'
gs_mag = 6

def get_scao_filename(npupil, gs_mag):
    return f'dfull/cube_Dfull_20251031_3600s_100ms_Kmag{gs_mag}_0piston_meters_scao_only_{npupil}.fits'

def get_wv_filename(band, npupil, rms, samp):
    return f'wv/cube_Cbasic_20210601_3600s_{samp}ms_0piston_meters_scao_only_{band}_{npupil}_WVonly_rms_{rms}.fits'


# --  Create WVonly screen and SCAO+WV screens
npupil = npupils[band]

# WV initial phase screens ('scao_only' is a mis-nomination)
wv = fits.getdata('wv/cube_Cbasic_20210601_3600s_100ms_0piston_meters_scao_only_720_WV.fits')    
scao = fits.getdata(get_scao_filename(npupil, gs_mag))

# Resize and rescale WV cubes
if band == 'L':
    wv_cube = resize_cube(wv, npupil) * wv_rms_L / temporal_rms
    rms = wv_rms_L
elif band == 'N2':
    wv_cube = resize_cube(wv, npupil) * wv_rms_L / temporal_rms
    rms = wv_rms_N

# save WVonly and scao+wv screens
wv_name = get_wv_filename(band, npupil, rms, samp) 
#f'wv/cube_Cbasic_20210601_3600s_{samp}ms_0piston_meters_scao_only_{band}_{npupil}_WVonly_rms_{rms}.fits''
scao_wv_name = f'dfull/cube_Dfull_20251031_3600s_{samp}ms_wv_ncpa_rms_{rms}_{band}_{npupil}.fits'

if samp == 100: # initial WV sampling is 100 ms
    fits.writeto(wv_name, wv_cube)
    fits.writeto(scao_wv_name, (scao + wv_cube))
elif samp == 300:
    fits.writeto(wv_name, wv_cube[::3])
    fits.writeto(scao_wv_name, (scao + wv_cube)[::3])
#----------------------------#

def load_ncpa(case, band, samp=100, rms=0, gs_mag=6):
    '''
    - rewritten for Dfull in November 2025
    '''
    npupil = npupils[band]
    if samp == 100:
        scao = fits.getdata(get_scao_filename(npupil, gs_mag))
        cbw = np.repeat(fits.getdata('cbw/20221006/ncpa/%s_%s.fits'%(tag[band], rep[band])), repeats=3, axis=0)
    elif samp == 300:
        print('NOT IMPLEMENTED')
    pup = fits.getdata(f'dfull/mask4heeps_Telescope_Pupil_{npupil}.fits')
    pup[pup < 0.5] = 0
    if 'cbw' in case:
        ncpa = scao + cbw
    else:
        wv = fits.getdata(get_wv_filename(band, npupil, rms, samp))
        if 'wv' in case:
            print('returning [scao + wv]')
            ncpa = scao + wv
        else: # ALL NCPA
            print('returning [all ncpa]')
            ncpa = scao + cbw + wv
    return pup, ncpa


def load_zpols_integ(pup, ncpa, ncpa_name, npupil, lag,
                     nLF, nHF, sigLF, sigHF,
                     nzer=nzer, G=0.4):
    '''
        nLF (int): frame decimation for tip-tilt
        nHF (int): frame decimation for higher modes
    '''
    zpols_integ_name = ncpa_name[:-5] + '_zpols_%s_nLF_%s_nHF_%s_G_%s_lag_%s_sigLF_%s_sigHF_%s.fits'%(nzer, freqLF, freqHF, G, lag, sigLF, sigHF)
    
    if os.path.isfile(zpols_integ_name):
        zpols_integ = fits.getdata(zpols_integ_name)
        print('  getdata ' + zpols_integ_name)
    else:
        print('  write to ' + zpols_integ_name)
        zpols = load_zpols(pup, ncpa, ncpa_name, npupil, nzer=nzer)
        zpols_integ = np.zeros(zpols.shape)
        nframes = len(zpols)
        # piston
        zpols_integ[:,0] = zpols[:,0]
        # tip-tilt, and higher modes
        for m, freq, sig in zip([range(1,3), range(3, nzer)], [freqLF, freqHF], [sigLF, sigHF]):
            if m.start == 3 and m.stop > 3:
                for n in range(freq+lag, nframes, freq):
                    error = np.mean(zpols[n-freq-lag:n-lag,m] - zpols_integ[n-freq-lag:n-lag,m], 0) + np.random.normal(0, sig, (1, len(m))) 
                    zpols_integ[n:n+freq,m] = zpols_integ[n-1,m] + G*error
        fits.writeto(zpols_integ_name, np.float32(zpols_integ))
    return zpols_integ

In [ ]:
# for case in ['wv', 'cbw', 'all']:
#     for band in ['L', 'N2']:
#         save_ncpa_cube(case, band, lag, freq, freq)